import dependencies

In [5]:
from faker import Faker
import mysql.connector
import random
from datetime import datetime, timedelta

intiat faker and connect to MySQL server

In [6]:
fake = Faker()

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="dhyey3074",
    database="amazon_prime_db"
)
cursor = conn.cursor()

populate users table

In [7]:
users = []
for _ in range(50):
    name = fake.name()
    email = fake.unique.email()
    password_hash = fake.sha256()
    signup_date = fake.date_between(start_date='-2y', end_date='today')
    subscription_id = random.randint(1, 3)
    users.append((name, email, password_hash, signup_date, subscription_id))

cursor.executemany("""
    INSERT INTO Users (name, email, password_hash, signup_date, subscription_id)
    VALUES (%s, %s, %s, %s, %s)
""", users)

populate subscription table

In [9]:
subscriptions = [
    ("Basic", 129.00, 1),
    ("Standard", 299.00, 3),
    ("Premium", 999.00, 12)
]

cursor.executemany("""
    INSERT INTO Subscriptions (plan_name, price, duration_months)
    VALUES (%s, %s, %s)
""", subscriptions)

populate genre table

In [10]:
genres = [
    "Action", "Drama", "Comedy", "Thriller", "Romance",
    "Horror", "Adventure", "Documentary", "Sci-Fi", "Crime"
]
cursor.executemany("INSERT INTO Genres (genre_name) VALUES (%s)", [(g,) for g in genres])

populate movies table

In [11]:
movies = []
for _ in range(100):
    title = fake.catch_phrase()
    release_year = random.randint(2000, 2025)
    duration_min = random.randint(60, 180)
    rating = random.choice(["U", "U/A", "A", "PG-13", "R"])
    movies.append((title, release_year, duration_min, rating))

cursor.executemany("""
    INSERT INTO Movies (title, release_year, duration_min, rating)
    VALUES (%s, %s, %s, %s)
""", movies)

Link movies to genre

In [12]:
cursor.execute("SELECT movie_id FROM Movies")
movie_ids = [row[0] for row in cursor.fetchall()]
genre_ids = list(range(1, len(genres) + 1))

movie_genres = []
for movie_id in movie_ids:
    assigned_genres = random.sample(genre_ids, random.randint(1, 3))
    for genre_id in assigned_genres:
        movie_genres.append((movie_id, genre_id))

cursor.executemany("""
    INSERT INTO MovieGenres (movie_id, genre_id)
    VALUES (%s, %s)
""", movie_genres)

insert into reviews table

In [13]:
reviews = []
for _ in range(200):
    user_id = random.randint(1, 50)
    movie_id = random.choice(movie_ids)
    rating = random.randint(1, 5)
    review_text = fake.sentence(nb_words=random.randint(8, 15))
    review_date = fake.date_between(start_date='-1y', end_date='today')
    reviews.append((user_id, movie_id, rating, review_text, review_date))

cursor.executemany("""
    INSERT INTO Reviews (user_id, movie_id, rating, review_text, review_date)
    VALUES (%s, %s, %s, %s, %s)
""", reviews)

insert into devices table

In [14]:
devices = []
device_types = ["Mobile", "Smart TV", "Laptop", "Tablet"]
for user_id in range(1, 51):
    for _ in range(random.randint(1, 2)):
        device_type = random.choice(device_types)
        device_name = fake.word().capitalize() + " " + random.choice(["Pro", "Plus", "Max", "HD"])
        last_used = fake.date_time_between(start_date='-2M', end_date='now')
        devices.append((user_id, device_type, device_name, last_used))

cursor.executemany("""
    INSERT INTO Devices (user_id, device_type, device_name, last_used)
    VALUES (%s, %s, %s, %s)
""", devices)

populate watch history table

In [15]:
for _ in range(500):
    user_id = random.randint(1, 50)
    movie_id = random.randint(1, 100)
    watch_date = fake.date_time_between(start_date='-6M', end_date='now')
    progress = random.randint(10, 100)
    cursor.execute("""
        INSERT INTO WatchHistory (user_id, movie_id, watch_date, progress_percentage)
        VALUES (%s, %s, %s, %s)
    """, (user_id, movie_id, watch_date, progress))

In [16]:
conn.commit()
cursor.close()
conn.close()
print("✅ Dummy data for all tables inserted successfully!")

✅ Dummy data for all tables inserted successfully!
